In [1]:
import numpy as np 
import pandas as pd
import pickle 

## Load Processed Vectorized Data

In [2]:
data = pickle.load(open('data_face_features.pickle', mode='rb'))

In [3]:
data.keys()

dict_keys(['data', 'label'])

## Splitting Data

In [4]:
X = np.array(data['data'])
y = np.array(data['label'])

In [5]:
X.shape,y.shape

((1674, 1, 128), (1674,))

In [6]:
X = X.reshape(-1,128)

In [7]:
X.shape

(1674, 128)

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=0)

In [10]:
X_train.shape,X_test.shape, y_train.shape, y_test.shape

((1339, 128), (335, 128), (1339,), (335,))

## Training Models

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

### Logistic Regression

In [12]:
model_log = LogisticRegression()
model_log.fit(X_train,y_train)

LogisticRegression()

In [13]:
y_pred_train = model_log.predict(X_train)
y_pred_test = model_log.predict(X_test)

#accuracy score
acc_train = accuracy_score(y_train,y_pred_train)
acc_test = accuracy_score(y_test,y_pred_test)

# f1-score 
f1_score_train = f1_score(y_train,y_pred_train,average='macro')
f1_score_test= f1_score(y_test,y_pred_test,average='macro')

### Logistic Regression Evaluation Metrics

In [14]:
print(f'accuracy score for training set = {acc_train}')
print(f'accuracy score for test set = {acc_test}')
print(f'f1 score for training set = {f1_score_train}')
print(f'f1 score score for test set = {f1_score_test}')

accuracy score for training set = 0.7214339058999253
accuracy score for test set = 0.6507462686567164
f1 score for training set = 0.6961108276565113
f1 score score for test set = 0.6128525329570255


In [15]:
#function to evaluate different trained models
def evaluation(model,x_train,y_train,x_test,y_test):
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    #accuracy score
    acc_train = accuracy_score(y_train,y_pred_train)
    acc_test = accuracy_score(y_test,y_pred_test)

    # f1-score 
    f1_score_train = f1_score(y_train,y_pred_train,average='macro')
    f1_score_test= f1_score(y_test,y_pred_test,average='macro')
    print(f'accuracy score for training set = {acc_train}')
    print(f'accuracy score for test set = {acc_test}')
    print(f'f1 score for training set = {f1_score_train}')
    print(f'f1 score score for test set = {f1_score_test}')

### SVM

In [16]:
model_svc = SVC(probability=True)
model_svc.fit(X_train,y_train)

SVC(probability=True)

In [17]:
evaluation(model_svc,X_train,y_train,X_test,y_test) #svc performs better than Log

accuracy score for training set = 0.8319641523525019
accuracy score for test set = 0.7074626865671642
f1 score for training set = 0.8325899156931432
f1 score score for test set = 0.7150649550951717


### Random Forest

In [18]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train,y_train)

RandomForestClassifier()

In [19]:
evaluation(model_rf,X_train,y_train,X_test,y_test) #overfit 

accuracy score for training set = 1.0
accuracy score for test set = 0.6626865671641791
f1 score for training set = 1.0
f1 score score for test set = 0.6598291568556367


### Voting Classifier 

In [20]:
model_voting = VotingClassifier(estimators=[
    ('log',LogisticRegression()),
    ('svm',SVC(probability=True)),
    ('rf',RandomForestClassifier())
], voting='soft', weights=[2,3,1])

In [21]:
model_voting.fit(X_train,y_train)

VotingClassifier(estimators=[('log', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier())],
                 voting='soft', weights=[2, 3, 1])

In [22]:
evaluation(model_voting,X_train,y_train,X_test,y_test)

accuracy score for training set = 0.878267363704257
accuracy score for test set = 0.7044776119402985
f1 score for training set = 0.8834800887190104
f1 score score for test set = 0.7216738259569547


In [23]:
from sklearn.model_selection import GridSearchCV

In [24]:
model_grid = GridSearchCV(model_voting,
                          param_grid={
                              'svm__C':[3,5,7,10],
                              'svm__gamma':[0.1,0.3,0.5],
                              'rf__n_estimators':[5,10,20],
                              'rf__max_depth':[3,5,7],
                              'voting':['soft','hard']
                              
                          },scoring='accuracy',cv=3,n_jobs=1,verbose=2)

In [26]:
model_grid.fit(X_train,y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.8s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   1.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   1.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   1.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   1.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   1.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.8s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   1.1s
[

[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=soft; total time=   1.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.9s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.9s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.7s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.8s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.9s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft; total time=   1.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft; total time=   1.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.9s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3

[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=soft; total time=   1.0s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=soft; total time=   1.0s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.9s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=hard; total time=   0.8s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=hard; total time=   1.1s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=hard; total time=   1.6s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.1, voting=soft; total time=   1.9s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.1, voting=soft; total time=   1.2s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.1, voting=soft; total time=   1.7s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5

[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.1, voting=soft; total time=   1.4s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.1, voting=soft; total time=   0.9s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.1, voting=hard; total time=   1.0s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.1, voting=hard; total time=   1.0s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.1, voting=hard; total time=   0.9s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.9s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.8s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.9s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.3, voting=hard; total time=   1.3s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gam

[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.9s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.9s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.8s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.9s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=soft; total time=   1.2s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=soft; total time=   1.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=soft; total time=   1.1s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=hard; total time=   1.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=hard; total time=   1.2s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5

[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=5, svm__gamma=0.5, voting=soft; total time=   1.1s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=5, svm__gamma=0.5, voting=hard; total time=   1.1s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=5, svm__gamma=0.5, voting=hard; total time=   0.8s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=5, svm__gamma=0.5, voting=hard; total time=   0.9s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=soft; total time=   1.1s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=soft; total time=   1.1s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=soft; total time=   1.0s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=hard; total time=   1.1s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=hard; total time=   0.9s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7

[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.1, voting=hard; total time=   1.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.1, voting=hard; total time=   1.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.1, voting=hard; total time=   0.7s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=soft; total time=   0.9s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=soft; total time=   1.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=soft; total time=   1.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=hard; total time=   0.9s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=hard; total time=   0.6s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=hard; total time=   0.7s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gam

[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.3, voting=hard; total time=   1.0s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.3, voting=hard; total time=   1.0s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=soft; total time=   0.6s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=soft; total time=   0.9s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=soft; total time=   0.9s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=hard; total time=   1.0s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=hard; total time=   1.1s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=hard; total time=   0.7s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=10, svm__gamma=0.1, voting=soft; total time=   0.8s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=

[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=7, svm__gamma=0.5, voting=hard; total time=   1.1s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=7, svm__gamma=0.5, voting=hard; total time=   1.0s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.1, voting=soft; total time=   1.0s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.1, voting=soft; total time=   1.0s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.1, voting=soft; total time=   0.8s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.1, voting=hard; total time=   1.0s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.1, voting=hard; total time=   1.1s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.1, voting=hard; total time=   1.1s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.3, voting=soft; total time=   1.1s
[CV] END rf__max_depth=7, rf__n_estimators=20, s

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('log',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier())],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [27]:
model_grid.best_params_

{'rf__max_depth': 5,
 'rf__n_estimators': 5,
 'svm__C': 10,
 'svm__gamma': 0.5,
 'voting': 'soft'}

In [28]:
#model_grid.best_estimator_

VotingClassifier(estimators=[('log', LogisticRegression()),
                             ('svm', SVC(C=10, gamma=0.5, probability=True)),
                             ('rf',
                              RandomForestClassifier(max_depth=5,
                                                     n_estimators=5))],
                 voting='soft', weights=[2, 3, 1])

In [29]:
model_grid.best_score_

0.7027484341716743

In [31]:
model_best_estimator = model_grid.best_estimator_

### Save Model

In [33]:
pickle.dump(model_best_estimator,open('models/machinelearning_face_person_identity2.pkl',mode='wb'))